In [1]:
import os
import cv2
import numpy as np
from tkinter import filedialog
from tkinter import Tk
from PIL import Image

def open_file_dialog():
    # Open dialog to select a file
    root = Tk()
    root.withdraw()  # Hide the root window
    filepath = filedialog.askopenfilename(title="Open image file")
    return filepath

def get_output_directory():
    # Open dialog to select output directory
    root = Tk()
    root.withdraw()
    directory = filedialog.askdirectory(title="Select output directory for MIB masks")
    return directory

def max_intensity_projection(image_stack):
    # Max Intensity Projection along Z-axis (axis=0)
    return np.max(image_stack, axis=0)

def apply_threshold(image, threshold):
    # Apply a binary threshold to the image
    _, mask = cv2.threshold(image, threshold, 255, cv2.THRESH_BINARY)
    return mask

def save_image_stack(image_stack, filename):
    # Save the image stack as a multi-page TIFF
    pil_images = [Image.fromarray(img) for img in image_stack]
    pil_images[0].save(filename, save_all=True, append_images=pil_images[1:], compression="tiff_deflate")
    
def process_image(image_path, output_dir, prefix):
    # Open the image stack (3D TIFF image)
    success, image_stack = cv2.imreadmulti(image_path, flags=cv2.IMREAD_ANYDEPTH)
    
    if not success:
        print("Failed to load the image stack.")
        return
    
    # Get max pixel intensity for threshold division
    max_intensity = int(np.max(image_stack))

    # Process each threshold level from 1 to max_intensity
    for i in range(1, max_intensity + 1):
        # Create an empty list to store masks for each slice
        mask_stack = []
        
        # Apply the threshold to each slice in the stack
        for image_slice in image_stack:
            mask = apply_threshold(image_slice, i)
            mask_stack.append(mask)

        # Save the stack of masks as a multi-page TIFF
        index = f"{i:02d}"  # Add leading zero for single-digit numbers
        output_path = os.path.join(output_dir, f"{prefix}_{index}.tif")
        save_image_stack(mask_stack, output_path)

        print(f"Saved: {output_path}")

def main():
    # Step 1: Open the image file
    image_path = open_file_dialog()
    
    # Step 2: Ask for the filename prefix
    prefix = input("Enter the desired filename prefix (default is 'default'): ") or "default"
    
    # Step 3: Select output directory
    output_dir = get_output_directory()
    
    # Step 4: Process the image and save masks
    process_image(image_path, output_dir, prefix)

if __name__ == "__main__":
    main()

2024-09-22 13:05:35.619 Python[4636:43774] WARNING: Secure coding is not enabled for restorable state! Enable secure coding by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState: and returning YES.


Saved: /Users/siyangc4/Downloads/default_00.tif
Saved: /Users/siyangc4/Downloads/default_01.tif
Saved: /Users/siyangc4/Downloads/default_02.tif
Saved: /Users/siyangc4/Downloads/default_03.tif
